In [1]:
import timeit
import sqlite3
import pandas as pd
from pandas.io import sql


start = timeit.default_timer()

# creating a connection object that represent the database
conn = sqlite3.connect('database.sqlite')

# representing the query string
query_train = """
select * from trainSearchStream limit 60;
"""

# Read SQl query into a dataframe 
df = sql.read_sql(query_train, conn)

print(df.head())

stop = timeit.default_timer()
total_time = stop - start

mins, secs = divmod(total_time, 60)
hours, mins = divmod(mins, 60)
print("------------Running Time is as below---------------------")
print("Total running time: %d:%d:%d.\n"  % (hours, mins, secs))

   SearchID      AdID  Position  ObjectType   HistCTR IsClick
0         2  11441863         1           3  0.001804       0
1         2  22968355         7           3  0.004723       0
2         3    212187         7           3  0.029701       0
3         3  34084553         1           3    0.0043       0
4         3  36256251         2           2                  
------------Running Time is as below---------------------
Total running time: 0:0:0.



##	Getting HistCTR and IsClick: Getting the historical Click through data and wheter the ad has been clicked. This is an important analysis since we can perform logistic regression on it. Further we can determine whether we can establish valuable relationships from the HistCTR and Click data 

In [2]:
start = timeit.default_timer()


query_One="""select HistCTR, IsClick from trainSearchStream limit 10000000 """
df_CTR=sql.read_sql(query_One,conn)
print(df_CTR.head())

stop = timeit.default_timer()
total_time = stop - start

mins, secs = divmod(total_time, 60)
hours, mins = divmod(mins, 60)
print("------------Running Time is as below-------------")
print("Total running time: %d:%d:%d.\n"  % (hours, mins, secs))


    HistCTR IsClick
0  0.001804       0
1  0.004723       0
2  0.029701       0
3    0.0043       0
4                  
------------Running Time is as below-------------
Total running time: 0:0:13.



## TrainSearchStream is an important table as it is been used for predictive modelling in this very project: One very important analysis will be Creating an upper level view of the search event by the user clicked on a particular ad. Now, since there are three tables involved here, we will have to perform nested joins. This table can be further imported to a csv file and we can apply gradient boosting to draw further conclusions. 

In [4]:
start = timeit.default_timer()

query_Two = """
SELECT * FROM
(
  SELECT * FROM TrainSearchStream tss
  LEFT OUTER JOIN SearchInfo si
  ON tss.SearchID = si.SearchID
  WHERE IsClick = 1
  LIMIT 2
) tssSubset
LEFT OUTER JOIN
AdsInfo ai
ON tssSubset.AdID = ai.AdID;
"""
df_Two = sql.read_sql(query_Two, conn)

print(df_Two.head())
    
stop = timeit.default_timer()
total_time = stop - start

mins, secs = divmod(total_time, 60)
hours, mins = divmod(mins, 60)
print("------------Running Time is as below-------------")
print("Total running time: %d:%d:%d.\n"  % (hours, mins, secs))

   SearchID      AdID  Position  ObjectType   HistCTR  IsClick  SearchID:1  \
0       121   1220223         7           3  0.005422        1         121   
1       181  13840581         7           3  0.013889        1         181   

              SearchDate     IPID   UserID    ...    LocationID  CategoryID  \
0  2015-05-09 22:21:40.0   490755  3817669    ...          2685          44   
1  2015-04-30 10:24:49.0  1507803  2391097    ...           230          22   

                                        SearchParams      AdID LocationID  \
0                                                      1220223              
1  {83:'Платья и юбки', 175:'Женская одежда', 90:...  13840581              

   CategoryID                                      Params  Price  \
0          44                 {223:'Бинокли и телескопы'}   1000   
1          22  {83:'Платья и юбки', 175:'Женская одежда'}   3000   

                                               Title IsContext  
0  Лупа,монокуляр на очко

### Along with finding looking for the reasons for more clicks, we can also do an analysis on data which is not causing clicks. For that, we can look for sample data from TrainSearchStream with searchinfo and adsinfo where the click is 0.

In [18]:
start = timeit.default_timer()

queryThree="""
SELECT * FROM
(
  SELECT * FROM TrainSearchStream tss
  LEFT OUTER JOIN SearchInfo si
  ON tss.SearchID = si.SearchID
  WHERE IsClick = 0
  LIMIT 2
) tssSubset
LEFT OUTER JOIN
AdsInfo ai
ON tssSubset.AdID = ai.AdID;
"""
df_three = sql.read_sql(queryThree, conn)
print(df_three.head())

stop = timeit.default_timer()
total_time = stop - start

mins, secs = divmod(total_time, 60)
hours, mins = divmod(mins, 60)
print("------------Running Time is as below-------------")
print("Total running time: %d:%d:%d.\n"  % (hours, mins, secs))

------------Running Time is as below-------------
Total running time: 0:7:49.



### The whole purpose of this project is to perform machine learning to leverage the advertising platform. This can be achieved when we can provide customized recommendations for the users of an ad platform. Having said, that, one important query can be to find out search event information just for a particular user. 

In [4]:
start = timeit.default_timer()

queryFour="""
select * from 
(VisitsStream vs inner join AdsInfo ai on vs.AdID = ai.AdID)
where UserId = 9;
"""
df_four = sql.read_sql(queryFour, conn)
print(df_four.head())

stop = timeit.default_timer()
total_time = stop - start

mins, secs = divmod(total_time, 60)
hours, mins = divmod(mins, 60)
print("------------Running Time is as below-------------")
print("Total running time: %d:%d:%d.\n"  % (hours, mins, secs))

   UserID    IPID      AdID               ViewDate      AdID  LocationID  \
0       9  885380  32207960  2015-04-27 20:16:36.0  32207960        1868   
1       9  885380  11013330  2015-04-27 20:18:00.0  11013330        1868   
2       9  885380  10699734  2015-04-27 20:18:21.0  10699734        1868   
3       9  885380  32207960  2015-04-27 20:32:05.0  32207960        1868   

   CategoryID                         Params Price  \
0      250005  {716:'Ремонт, строительство'}     0   
1      250005  {716:'Ремонт, строительство'}     0   
2      250005  {716:'Ремонт, строительство'}     0   
3      250005  {716:'Ремонт, строительство'}     0   

                                              Title  IsContext  
0      Забор из металлического штакетника в Талдоме          0  
1  Заборы из штакетника в Талдоме, Кимрах, Дмитрове          0  
2                    Монтаж заборов кровли сайдинга          0  
3      Забор из металлического штакетника в Талдоме          0  
------------Running Tim

### 5)	In the database, there are three types of ads : - Regular, Highlighted and Context. Context ads are the best way to target users. Currently, many advertisement companies uses general statistics on ad performance to drive the placement of the ads. Thus to concentrate more on context ads it is essential to filter those from the database.  

In [3]:
start = timeit.default_timer()

queryFive="""
SELECT * FROM
(
  SELECT * FROM TestSearchStream tss
  LEFT OUTER JOIN SearchInfo si
  ON tss.SearchID = si.SearchID
  WHERE ObjectType = 3
  LIMIT 10
) tssSubset
LEFT OUTER JOIN
AdsInfo ai
ON tssSubset.AdID = ai.AdID;
"""
df_five = sql.read_sql(queryFive, conn)
print(df_five.head())

stop = timeit.default_timer()
total_time = stop - start

mins, secs = divmod(total_time, 60)
hours, mins = divmod(mins, 60)
print("------------Running Time is as below-------------")
print("Total running time: %d:%d:%d.\n"  % (hours, mins, secs))

   TestId  SearchID      AdID  Position  ObjectType   HistCTR  SearchID:1  \
0       1         1  10915336         1           3  0.004999           1   
1       5         1  31173651         7           3  0.045094           1   
2       6         7   9041972         1           3  0.007032           7   
3       7        28  14309446         7           3  0.016943          28   
4       8        28  15753167         1           3  0.054339          28   

              SearchDate     IPID   UserID    ...    LocationID  CategoryID  \
0  2015-05-18 19:54:32.0  1717090  3640266    ...          1729           5   
1  2015-05-18 19:54:32.0  1717090  3640266    ...          1729           5   
2  2015-05-14 23:07:27.0  1849117  3434614    ...            44      500001   
3  2015-05-16 20:19:47.0  1845070   682366    ...          1338          12   
4  2015-05-16 20:19:47.0  1845070   682366    ...          1338          12   

   SearchParams      AdID LocationID  CategoryID  \
0         